In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
import torchvision as p
import torchvision.transforms as transforms
import os
import time
import json
import cv2 as cv
import argparse

In [3]:
import pandas as pd
cp = pd.read_csv('/home/haoying/res_zl12/center2PA.csv')
# df = pd.read_csv('nightlight_predicted_all.csv')
df = pd.read_csv('/home/haoying/res_zl12_effnet_v3/nightlight_predicted_all.csv')
df.columns = ['name','cluster_lat','cluster_lon','nightlights','predict']

In [25]:
# pd.merge(df2,df1,left_on='y_x',right_on='name')

In [4]:
df

,name,cluster_lat,cluster_lon,nightlights,predict
0,-48.779296875_-66.93006025862447.png,43.771094,90.966797,0.379018,4.194642
1,-44.82421875_-75.97355295343337.png,40.847060,120.058594,0.380550,4.330161
2,-44.82421875_-70.98834922412489.png,40.847060,102.304688,0.272340,4.132349
3,-39.111328125_-65.4034447883078.png,36.385913,87.275391,0.390618,4.072864
4,-50.80078125_-77.86034459764656.png,45.213004,128.320312,0.310670,4.083563
...,...,...,...,...,...
131947,-38.671875_-69.28725695167886.png,36.031332,97.382812,0.272444,4.591633
131948,-46.142578125_-72.63337363853837.png,41.836828,107.666016,0.306109,4.190571
131949,-44.208984375_-66.08936427047087.png,40.380028,88.945312,0.306339,4.150032
131950,-25.224609375_-74.59010800882324.png,24.447150,114.609375,0.263865,4.114062


In [5]:
df1 = pd.merge(df,cp,how='left')

In [6]:
score = df1.groupby('PAC').agg({'predict':'mean','name':'count','nightlights':'mean'})
score = pd.DataFrame(score).reset_index()
score.columns = ['PAC','predict','pic_num','nightlights']

In [7]:
import geopandas
import json

county_shp = geopandas.GeoDataFrame.from_file('/home/haoying/data/get_shp/县级行政区/县级行政区.shp')
county_shp['ID'] = [i for i in range(county_shp.shape[0])]
county_shp['parent'] = county_shp['parent'].apply(lambda x:json.loads(x).get('adcode'))
county_shp['adcode'] = county_shp['adcode'].astype(str)
county_shp.head()

ERROR 1: PROJ: proj_create_from_database: Open of /home/haoying/anaconda3/envs/torch/share/proj failed
PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


,adcode,name,childrenNu,level,parent,subFeature,geometry,ID
0,110101,东城区,0,district,110000,NaN,"POLYGON ((116.44364 39.87285, 116.44351 39.872...",0
1,110102,西城区,0,district,110000,NaN,"POLYGON ((116.32581 39.89679, 116.32581 39.897...",1
2,110105,朝阳区,0,district,110000,NaN,"MULTIPOLYGON (((116.59555 40.01752, 116.59820 ...",2
3,110106,丰台区,0,district,110000,NaN,"POLYGON ((116.25909 39.89667, 116.26005 39.896...",3
4,110107,石景山区,0,district,110000,NaN,"POLYGON ((116.16703 39.88876, 116.16636 39.889...",4


In [8]:
res1 = county_shp.set_index('adcode').join(score.set_index('PAC'))

In [9]:
res1 = res1.reset_index()
res1

,adcode,name,childrenNu,level,parent,subFeature,geometry,ID,predict,pic_num,nightlights
0,110101,东城区,0,district,110000,NaN,"POLYGON ((116.44364 39.87285, 116.44351 39.872...",0,NaN,NaN,NaN
1,110102,西城区,0,district,110000,NaN,"POLYGON ((116.32581 39.89679, 116.32581 39.897...",1,6.000537,1.0,44.620304
2,110105,朝阳区,0,district,110000,NaN,"MULTIPOLYGON (((116.59555 40.01752, 116.59820 ...",2,6.000586,2.0,22.427105
3,110106,丰台区,0,district,110000,NaN,"POLYGON ((116.25909 39.89667, 116.26005 39.896...",3,NaN,NaN,NaN
4,110107,石景山区,0,district,110000,NaN,"POLYGON ((116.16703 39.88876, 116.16636 39.889...",4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2843,659006,铁门关市,0,city,650000,NaN,"MULTIPOLYGON (((85.74147 41.75532, 85.74277 41...",2843,4.520817,10.0,0.549276
2844,659007,双河市,0,city,650000,NaN,"MULTIPOLYGON (((82.20696 44.82122, 82.17847 44...",2844,4.529892,11.0,0.764995
2845,659008,可克达拉市,0,city,650000,NaN,"MULTIPOLYGON (((80.67237 43.84470, 80.66106 43...",2845,4.777022,4.0,0.505605
2846,659009,昆玉市,0,city,650000,NaN,"MULTIPOLYGON (((79.57344 37.38167, 79.57990 37...",2846,4.342645,12.0,0.469621


In [10]:
eco = pd.read_csv('2017eco.csv')
eco['adcode'] = eco['adcode'].apply(lambda x :str(x)[:6])

In [11]:
res = pd.merge(res1,eco,on = 'adcode',how='left')

In [12]:
res.columns

Index(['adcode', 'name', 'childrenNu', 'level', 'parent', 'subFeature',
       'geometry', 'ID', 'predict', 'pic_num', 'nightlights', '县', '户籍人口数（万人）',
       '地区生产总值（万元）', '第一产业增加值（万元）', '第二产业增加值（万元）',
       '地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”）',
       '地方财政一般预算支出（万元）（2003年及之前为“财政总支出”）', '城乡居民储蓄存款余额（万元）',
       '年末金融机构各项贷款余额（万元）', '医院、卫生院床位数（床）', 'province', 'city', 'district'],
      dtype='object')

In [13]:
for i in res.columns[12:-3]:
    print('全国',i,res[['nightlights',i]].corr('spearman').iloc[1,0],res[['predict',i]].corr('spearman').iloc[1,0])

全国 户籍人口数（万人） 0.47824060926899525 0.4745908167776673
全国 地区生产总值（万元） 0.7056159706323019 0.6761452163745629
全国 第一产业增加值（万元） 0.27749158220465947 0.3066223690764618
全国 第二产业增加值（万元） 0.6925757014604601 0.6618887809249444
全国 地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”） 0.6513976926646571 0.6030385912674076
全国 地方财政一般预算支出（万元）（2003年及之前为“财政总支出”） 0.4114576378883565 0.3902970043463553
全国 城乡居民储蓄存款余额（万元） 0.6986688246356598 0.6810398597918889
全国 年末金融机构各项贷款余额（万元） 0.6473324881809971 0.6021341321168417
全国 医院、卫生院床位数（床） 0.5568845322373226 0.529818468201841


In [14]:
## 分地区

In [15]:
res.head()

,adcode,name,childrenNu,level,parent,subFeature,geometry,ID,predict,pic_num,...,第一产业增加值（万元）,第二产业增加值（万元）,地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”）,地方财政一般预算支出（万元）（2003年及之前为“财政总支出”）,城乡居民储蓄存款余额（万元）,年末金融机构各项贷款余额（万元）,医院、卫生院床位数（床）,province,city,district
0,110101,东城区,0,district,110000,NaN,"POLYGON ((116.44364 39.87285, 116.44351 39.872...",0,NaN,NaN,...,NaN,825900.0,1721700.0,2438300.0,NaN,NaN,NaN,北京市,北京市,东城区
1,110102,西城区,0,district,110000,NaN,"POLYGON ((116.32581 39.89679, 116.32581 39.897...",1,6.000537,1.0,...,NaN,3312900.0,4221200.0,4307700.0,NaN,NaN,NaN,北京市,北京市,西城区
2,110105,朝阳区,0,district,110000,NaN,"MULTIPOLYGON (((116.59555 40.01752, 116.59820 ...",2,6.000586,2.0,...,12200.0,4227200.0,5081700.0,5133800.0,NaN,NaN,NaN,北京市,北京市,朝阳区
3,110106,丰台区,0,district,110000,NaN,"POLYGON ((116.25909 39.89667, 116.26005 39.896...",3,NaN,NaN,...,7400.0,2816700.0,1131100.0,2274100.0,NaN,NaN,NaN,北京市,北京市,丰台区
4,110107,石景山区,0,district,110000,NaN,"POLYGON ((116.16703 39.88876, 116.16636 39.889...",4,NaN,NaN,...,NaN,1560700.0,564400.0,986500.0,NaN,NaN,NaN,北京市,北京市,石景山区


In [16]:
prov_code = {'11':'北京市','13':'河北省','14':'山西省','15':'内蒙古自治区','21':'辽宁省','22':'吉林省','23':'黑龙江省',
            '32':'江苏省','33':'浙江省','34':'安徽省','35':'福建省','36':'江西省','37':'山东省',
             '41':'河南省','42':'湖北省','43':'湖南省','44':'广东省','45':'广西壮族自治区','46':'海南省',
            '51':'四川省','52':'贵州省','53':'云南省','54':'西藏自治区',
            '61':'陕西省','62':'甘肃省','63':'青海省','64':'宁夏回族自治区','65':'新疆维吾尔自治区'}

test = res[(pd.notnull(res['predict'])&pd.isnull(res['province']))]
for a,b in zip(test.name, test.parent): 
    prov = prov_code[str(b)[:2]]
    res.loc[res['name']==a,'province'] = prov

In [17]:
# 华东地区包括山东省，江苏省，浙江省，安徽省，江西省，福建省，上海市；
# 华中地区包括湖北省，河南省，湖南省；
# 华南包括广东省，广西壮族自治区，海南省，香港特别行政区，澳门特别行政区；
# 华北包括山西省，河北省，北京市，天津市，内蒙古自治区；
# 东北地区包括吉林省，黑龙江省，辽宁省；
# 西北包括青海省，宁夏回族自治区，陕西省，甘肃省，新疆维吾尔自治区；
# 西南包括贵州省，云南省，重庆市，四川省，西藏自治区。
def get_diqu(x):
    if x in ['山东省','江苏省','浙江省','安徽省','江西省','福建省','上海市']:
        return '华东'
    elif x in ['广东省','广西壮族自治区','海南省','香港特别行政区','澳门特别行政区']:
        return '华南'
    elif x in ['内蒙古自治区','天津市','北京市','河北省','山西省']:
        return '华北'
    elif x in ['辽宁省','吉林省','黑龙江省']:
        return '东北'
    elif x in ['湖北省','河南省','湖南省']:
        return '华中'
    elif x in ['青海省','宁夏回族自治区','陕西省','甘肃省','新疆维吾尔自治区']:
        return '西北'
    elif x in ['贵州省','云南省','重庆市','四川省','西藏自治区']:
        return '西南'
    else:
        return None

In [18]:
# def get_diqu(x):
#     if x in ['北京市','天津市','河北省','上海市','江苏省','浙江省','福建省','山东省','广东省','海南省']:
#         return '东部'
#     elif x in ['山西省','安徽省','江西省','河南省','湖北省','湖南省']:
#         return '中部'
#     elif x in ['内蒙古自治区','广西壮族自治区','重庆市','四川省','贵州省','云南省','西藏自治区','陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区']:
#         return '西部'
#     elif x in ['辽宁省','吉林省','黑龙江省']:
#         return '东北'
#     else:
#         return None

In [19]:
res['地区'] = res['province'].apply(lambda x : get_diqu(x))

In [20]:
res.columns

Index(['adcode', 'name', 'childrenNu', 'level', 'parent', 'subFeature',
       'geometry', 'ID', 'predict', 'pic_num', 'nightlights', '县', '户籍人口数（万人）',
       '地区生产总值（万元）', '第一产业增加值（万元）', '第二产业增加值（万元）',
       '地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”）',
       '地方财政一般预算支出（万元）（2003年及之前为“财政总支出”）', '城乡居民储蓄存款余额（万元）',
       '年末金融机构各项贷款余额（万元）', '医院、卫生院床位数（床）', 'province', 'city', 'district',
       '地区'],
      dtype='object')

In [23]:
for i in ['华东','华中','华南','华北','东北','西北','西南']:
    res_dis = res[res['地区']==i]
    for j in res_dis.columns[12:-4]:
        print(i,j,res_dis[['nightlights',j]].corr('spearman').iloc[1,0],res_dis[['predict',j]].corr('spearman').iloc[1,0])

华东 户籍人口数（万人） 0.2985003221531914 0.3132182101966531
华东 地区生产总值（万元） 0.7906892764453791 0.7529475147663665
华东 第一产业增加值（万元） 0.3242172319393985 0.35692449492503797
华东 第二产业增加值（万元） 0.7599086974504664 0.7160771184711623
华东 地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”） 0.6809221367427609 0.6362205887883523
华东 地方财政一般预算支出（万元）（2003年及之前为“财政总支出”） 0.5090801271517977 0.491635386685141
华东 城乡居民储蓄存款余额（万元） 0.7515990698411995 0.729376063173097
华东 年末金融机构各项贷款余额（万元） 0.7625536342318437 0.6904678696048082
华东 医院、卫生院床位数（床） 0.5995698343112987 0.6167515516615976
华中 户籍人口数（万人） 0.09552246665528026 0.14879897066857692
华中 地区生产总值（万元） 0.4713857236621155 0.42356805647365336
华中 第一产业增加值（万元） -0.12954196966372364 -0.049226851539172174
华中 第二产业增加值（万元） 0.5185834014056516 0.48578714413305624
华中 地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”） 0.4129342116825924 0.33967479289744473
华中 地方财政一般预算支出（万元）（2003年及之前为“财政总支出”） 0.12147513069447177 0.10539328606443688
华中 城乡居民储蓄存款余额（万元） 0.3813167349618369 0.3417661571475366
华中 年末金融机构各项贷款余额（万元） 0.3686195648288789 0.28

In [24]:
## 修正
res['area'] = res['geometry'].apply(lambda x:x.area)

In [25]:
res.columns

Index(['adcode', 'name', 'childrenNu', 'level', 'parent', 'subFeature',
       'geometry', 'ID', 'predict', 'pic_num', 'nightlights', '县', '户籍人口数（万人）',
       '地区生产总值（万元）', '第一产业增加值（万元）', '第二产业增加值（万元）',
       '地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”）',
       '地方财政一般预算支出（万元）（2003年及之前为“财政总支出”）', '城乡居民储蓄存款余额（万元）',
       '年末金融机构各项贷款余额（万元）', '医院、卫生院床位数（床）', 'province', 'city', 'district',
       '地区', 'area'],
      dtype='object')

In [27]:
res['predict_adj'] = res['predict']*res['area']
for i in res.columns[12:-6]:
    print('全国',i,res[['nightlights',i]].corr('spearman').iloc[1,0],res[['predict',i]].corr('spearman').iloc[1,0],res[['predict_adj',i]].corr('spearman').iloc[1,0])

全国 户籍人口数（万人） 0.47824060926899525 0.4745908167776673 -0.15223614796394414
全国 地区生产总值（万元） 0.7056159706323019 0.6761452163745629 -0.3432091914640039
全国 第一产业增加值（万元） 0.27749158220465947 0.3066223690764618 0.07169895647940812
全国 第二产业增加值（万元） 0.6925757014604601 0.6618887809249444 -0.3797113012685811
全国 地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”） 0.6513976926646571 0.6030385912674076 -0.30109363221095803
全国 地方财政一般预算支出（万元）（2003年及之前为“财政总支出”） 0.4114576378883565 0.3902970043463553 0.011839426384515796
全国 城乡居民储蓄存款余额（万元） 0.6986688246356598 0.6810398597918889 -0.32376414445924057
全国 年末金融机构各项贷款余额（万元） 0.6473324881809971 0.6021341321168417 -0.2312141624727623
全国 医院、卫生院床位数（床） 0.5568845322373226 0.529818468201841 -0.23092432200334007


In [24]:
for i in ['华东','华中','华南','华北','东北','西北','西南']:
    res_dis = res[res['地区']==i]
    for j in res_dis.columns[12:-7]:
        print(i,j,res_dis[['nightlights',j]].corr('spearman').iloc[1,0],res_dis[['predict',j]].corr('spearman').iloc[1,0],res_dis[['predict_adj',j]].corr('spearman').iloc[1,0])

华东 户籍人口数（万人） 0.2985003221531914 0.3132182101966531 0.3893714884282948
华东 地区生产总值（万元） 0.7906892764453791 0.7529475147663665 -0.0482923565790981
华东 第一产业增加值（万元） 0.3242172319393985 0.35692449492503797 0.40459251933380874
华东 第二产业增加值（万元） 0.7599086974504664 0.7160771184711623 -0.06887746558020912
华东 地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”） 0.6809221367427609 0.6362205887883523 -0.04705905129739464
华东 地方财政一般预算支出（万元）（2003年及之前为“财政总支出”） 0.5090801271517977 0.491635386685141 0.2624714833522937
华东 城乡居民储蓄存款余额（万元） 0.7515990698411995 0.729376063173097 0.047530088897819116
华东 年末金融机构各项贷款余额（万元） 0.7625536342318437 0.6904678696048082 0.040097671406852484
华中 户籍人口数（万人） 0.09552246665528026 0.14879897066857692 0.38965687005553795
华中 地区生产总值（万元） 0.4713857236621155 0.42356805647365336 -0.029514013852442473
华中 第一产业增加值（万元） -0.12954196966372364 -0.049226851539172174 0.5318609112694248
华中 第二产业增加值（万元） 0.5185834014056516 0.48578714413305624 -0.12823233599171474
华中 地方财政一般预算收入（万元）（2003年及之前为“地方财政预算内收入”） 0.4129342116825924 0.339

## 泰尔指数

In [25]:
res.dropna(subset=['predict'], inplace=True)

In [26]:
def Normalization(x):
    return [(float(i)-np.min(x))/float(np.max(x)-np.min(x)) for i in x]
res['predict_norm'] = Normalization(res['predict'])

In [27]:
# 全国
import math
def get_theil(x):
    n = len(x)
    x_avg = np.mean(x)
    t_x = [(i/x_avg)*math.log(i/x_avg) if i !=0 else i for i in x ]
    t = sum(t_x)/n
    return t
get_theil(res['predict_norm'])

0.23694545147358131

In [28]:
# 中东西
t_in = 0
t_out = 0
for i in ['华东','华中','华南','华北','东北','西北','西南']:
    res_dis = res[res['地区']==i]
    # 各地区theil指数
    t_i = get_theil(res_dis['predict_norm'])
#     print(t_i)
#     print(i,get_theil(res_dis['predict']))
    # 样本个数之比
    num_p = len(res_dis)/len(res)
#     print(num_p)
    # 指数和之比
    index_p = sum(res_dis['predict_norm']) /sum(res['predict_norm'])
    ## 区域内差异
    t_in += t_i*index_p
    ## 区域间差异
    t_out += index_p*math.log(index_p/num_p)
    ## 贡献率
    p = t_i * index_p /get_theil(res['predict_norm'])
    
    print(i,t_i,p)
print('区域内',t_in,'区域间',t_out)
print(t_in/get_theil(res['predict_norm']),t_out/get_theil(res['predict_norm']))

华东 0.15153614291842082 0.1867373512594059
华中 0.1716801627172768 0.1031028090429976
华南 0.25485055571459614 0.09503371980321751
华北 0.15741433014546974 0.11477519803312919
东北 0.2070770352365758 0.0843761217079473
西北 0.21563828814419897 0.09064983424745482
西南 0.3190129221495109 0.14600066064488698
区域内 0.19445537300333665 区域间 0.042490078470245135
0.8206756947390392 0.17932430526096277


In [29]:
# 分省
# 中东西

for i in ['华东','华中','华南','华北','东北','西北','西南']:
    res_dis = res[res['地区']==i]
    t_in = 0
    t_out = 0
    p_all = 0
    for j in res_dis['province'].unique():
        pro_data = res_dis[res_dis['province']==j]
        # 各地区theil指数
        t_i = get_theil(pro_data['predict_norm'])
        # 样本个数之比
        num_p = len(pro_data)/len(res_dis)
        # 指数和之比
        index_p = sum(pro_data['predict_norm']) /sum(res_dis['predict_norm'])
        ## 区域内差异
        t_in += t_i*index_p
        ## 区域间差异
        t_out += index_p*math.log(index_p/num_p)
        ## 贡献率
        p = t_i * index_p /get_theil(res_dis['predict_norm'])
        p_all += p
        print(j,t_i,p)
    
    print(i,'区域内',t_in,'区域间',t_out)
    print(p_all)
    print(t_in/get_theil(res[res['地区']==i]['predict_norm']),t_out/get_theil(res[res['地区']==i]['predict_norm']))

上海市 0.013557775819407156 0.003944784146843738
江苏省 0.05479093408073915 0.08087234617719162
浙江省 0.1428113550648807 0.14986522035709657
安徽省 0.10007291483986855 0.09941801406767461
福建省 0.2378332981548313 0.11499493192583599
江西省 0.17444846660743107 0.10976286620512882
山东省 0.04639832888085677 0.07778907786599569
华东 区域内 0.09647506726226883 区域间 0.05506107565615209
0.636647240745767
0.6366472407457671 0.36335275925423355
河南省 0.09105082688588036 0.2952363365982724
湖北省 0.1790431172964841 0.23461582294971614
湖南省 0.17097267428782184 0.21745215045951902
华中 区域内 0.12829732554141116 区域间 0.04338283717586547
0.7473043100075076
0.7473043100075075 0.2526956899924915
广东省 0.20939960900189555 0.49022531162757566
海南省 0.14631775569519728 0.06089341759500725
广西壮族自治区 0.19203215656418834 0.2240240095533276
华南 区域内 0.19754555773517482 区域间 0.05730499797942157
0.7751427387759104
0.7751427387759106 0.22485726122409047
北京市 0.13341119944439622 0.04685156606433567
天津市 0.038673711078984624 0.011297833882698378
河北省 0.085118